Imports

In [1]:
import kagglehub
import os
import pandas as pd
import numpy as np
import re

c:\Users\jakse\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\jakse\AppData\Local\Temp\ipykernel_13276\4051799869.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Data Downloads

In [2]:
# Germany
path_germany = kagglehub.dataset_download("corrieaar/apartment-rental-offers-in-germany")
germany_csvs = [f for f in os.listdir(path_germany) if f.endswith('.csv')]
df_germany = pd.read_csv(os.path.join(path_germany, germany_csvs[0]))

# Poland
path_poland = kagglehub.dataset_download("krzysztofjamroz/apartment-prices-in-poland")
poland_csvs = [f for f in os.listdir(path_poland) if f.endswith('.csv')]

# Filter rental files only
rental_files = [f for f in poland_csvs if 'apartments_rent_pl' in f.lower()]

# Load and combine with date column extracted from filename
poland_frames = []

for f in rental_files:
    match = re.search(r'(\d{4})_(\d{2})', f)
    if match:
        year, month = match.groups()
        file_date = f'{year}-{month}-01'  # standardized to first of month

        df = pd.read_csv(os.path.join(path_poland, f))
        df['date'] = pd.to_datetime(file_date)
        poland_frames.append(df)

# Final combined DataFrame
df_poland = pd.concat(poland_frames, ignore_index=True)

Data Processing Pipeline
comments:

-only germany has heatingType data

-description not needed imo

-dates not needed imo

-added floor and elevator data and they both had it, and they could be interesting

In [ ]:
# Germany
selected_columns_germany = ['livingSpace', 'noRooms', 'balcony', 'heatingType', 'yearConstructed',
                           'totalRent', 'baseRent', 'serviceCharge', 'heatingCosts', 'date', 'floor', 'lift'] 
df_germany_selected = df_germany[selected_columns_germany]

# Fill missing totalRent with baseRent + serviceCharge + heatingCosts
df_germany['totalRent'] = df_germany['totalRent'].fillna(
    df_germany['baseRent'].fillna(0) + df_germany['serviceCharge'].fillna(0) + df_germany['heatingCosts'].fillna(0)
)

df_germany['date'] = pd.to_datetime(df_germany['date'], format='%b%y')

# Select and rename columns
df_germany_cleaned = pd.DataFrame({
    'size': df_germany['livingSpace'],
    'number_of_rooms': df_germany['noRooms'],
    'construction_year': df_germany['yearConstructed'],
    'total_rent': df_germany['totalRent'],
    'floor': df_germany['floor'],
    'balcony': df_germany['balcony'],
    'elevator': df_germany['lift'],
    'date': df_germany['date'],
    'country': 'Germany'
})

# Poland 
selected_columns_poland = ['squareMeters', 'rooms', 'hasBalcony', 'buildYear', 'price', 'floor', 'hasElevator']
df_poland_selected = df_poland[selected_columns_poland]

# Exchange rate: 1 Złoty = 0.23 Euro
zloty_euro_rate = 0.23
df_poland['price_eur'] = df_poland['price'] * zloty_euro_rate

# format balcony and elevator
df_poland['hasBalcony'] = df_poland['hasBalcony'].map({"yes": True, "no": False})
df_poland['hasElevator'] = df_poland['hasElevator'].map({"yes": True, "no": False})

# Select and rename columns
df_poland_cleaned = pd.DataFrame({
    'size': df_poland['squareMeters'],
    'number_of_rooms': df_poland['rooms'],
    'construction_year': df_poland['buildYear'],
    'total_rent': df_poland['price_eur'],
    'floor': df_poland['floor'],
    'balcony': df_poland['hasBalcony'],
    'elevator': df_poland['hasElevator'],
    'date': df_poland['date'],
    'country': 'Poland'
})

# Combine both datasets
df_combined_rentals = pd.concat([df_germany_cleaned, df_poland_cleaned], ignore_index=True)

# Cleaning Data
df_combined_rentals = df_combined_rentals[
    (df_combined_rentals['total_rent'] >= 300) &
    (df_combined_rentals['number_of_rooms'] <= 10) &
    (df_combined_rentals['total_rent'] <= 20_000)
]

df_combined_rentals = df_combined_rentals[df_combined_rentals['number_of_rooms'] % 1 == 0]

# save to csv
df_combined_rentals.to_csv('data/cleaned_apartments_rentals.csv', index=False)

Dimensional Model

In [4]:
# Country Dimension
dim_country = pd.DataFrame({
    'country': ['Germany', 'Poland'],
    'currency': ['EUR', 'PLN'],
    'purchasing_power_parity': [1.0, 0.8]
})
dim_country['countryKey'] = dim_country.index + 1

# Apartment Dimension
dim_apartment = df_combined_rentals[['size', 'number_of_rooms', 'balcony', 'construction_year', 'elevator', 'floor']].drop_duplicates().reset_index(drop=True)

dim_apartment['apartmentKey'] = dim_apartment.index + 1


# Date Dimension
df_combined_rentals['date'] = pd.to_datetime(df_combined_rentals['date'])

dim_date = df_combined_rentals[['date']].drop_duplicates().reset_index(drop=True)

dim_date['dateKey'] = dim_date.index + 1

dim_date = dim_date[['dateKey', 'date']]


# Rent Range Dimension
bins = [0, 500, 1000, 1500, 2000, float('inf')]
labels = ['<500', '500-1000', '1000-1500', '1500-2000', '>2000']
df_combined_rentals['rentRange'] = pd.cut(df_combined_rentals['total_rent'], bins=bins, labels=labels)

dim_rent_category = pd.DataFrame({
    'rentRange': df_combined_rentals['rentRange'].dropna().unique()
})
dim_rent_category['rentCategoryKey'] = dim_rent_category.index + 1


# join Fact Table
df_fact = df_combined_rentals.merge(dim_country, on='country', how='left')
df_fact = df_fact.merge(dim_apartment, on=['size', 'number_of_rooms', 'balcony', 'construction_year', 'elevator', 'floor'], how='left')
df_fact = df_fact.merge(dim_date, on='date', how='left')
df_fact = df_fact.merge(dim_rent_category, on='rentRange', how='left')

fact_offer = df_fact[['countryKey', 'apartmentKey', 'rentCategoryKey', 'dateKey', 'total_rent']]
fact_offer = fact_offer.rename(columns={'total_rent': 'rentAmount'})


output_dir = 'data/dimensional'
os.makedirs(output_dir, exist_ok=True)

dim_country.to_csv(f'{output_dir}/dim_country.csv', index=False)
dim_apartment.to_csv(f'{output_dir}/dim_apartment.csv', index=False)
dim_rent_category.to_csv(f'{output_dir}/dim_rent_category.csv', index=False)
dim_date.to_csv(f'{output_dir}/dim_date.csv', index=False)

fact_offer.to_csv(f'{output_dir}/fact_offer.csv', index=False)